In [ ]:
import sys
import asyncio
import time

import numpy as np

from lsst.ts import salobj

from lsst.ts.standardscripts.auxtel.attcs import ATTCS
from lsst.ts.standardscripts.auxtel.latiss import LATISS

In [ ]:
# Temporarily needed to run at summit
import os
os.environ["LSST_DDS_DOMAIN"] = 'lsatmcs'
os.environ["OSPL_URI"] = "file:///home/patrickingraham/ospl.xml"

In [ ]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
#get classes and start them
domain = salobj.Domain()
attcs = ATTCS(domain)
latiss = LATISS(domain)
await asyncio.gather(attcs.start_task, latiss.start_task)

In [5]:
# enable components
#await attcs.enable()
#await latiss.enable()

In [6]:
# take event checking out the slew commands to test telescope only
attcs.check.atdome = False
attcs.check.atdometrajectory = False

In [7]:
# send telescope to alt/az/
#await salobj.set_summary_state(latiss.atcamera, salobj.State.ENABLED)

RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 33 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 49 messages


In [8]:
# point telescope to desired starting position
# this make take extra time as more checks are performed
start_az=50
start_el=20
start_rot_pa=0
await attcs.point_azel(start_az, start_el, rot_pa=start_rot_pa, wait_dome=True)

Sending command
Stop tracking.
Mount tracking state is 9
Tracking state: 10.
Tracking state: 8.
In Position: True.
In Position: False.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
[Telescope] delta Alt = -038.837 | delta Az = -000.581 delta N1 = +000.000 delta N2 = +032.843 [Dome] delta Az = +003.060
[Telescope] delta Alt = -035.585 | delta Az = -000.001 delta N1 = +000.000 delta N2 = +027.621 [Dome] delta Az = +003.060
[Telescope] delta Alt = -029.588 | delta Az = +000.000 delta N1 = +000.000 delta N2 = +021.717 [Dome] delta Az = +003.060
[Telescope] delta Alt = -023.624 | delta Az = +000.000 delta N1 = +000.000 delta N2 = +016.140 [Dome] delta Az = +003.060
[Telescope] delta Alt = -017.847 | delta Az = +000.000 delta N1 = +000.000 delta N2 = +011.128 [Dome] delta Az = +003.06

In [9]:
#declare offset size
d_az= 3.5 # degrees
d_el = 3.5 # degrees
d_rot= 3.5 # degrees

d_az= 10 # degrees
d_el = 10 # degrees
d_rot= 10 # degrees

d_az= 90 # degrees
d_el = 60 # degrees
d_rot= 90 # degrees

In [10]:
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [11]:
# get RA/DEC of current position
az = Angle(start_az, unit=u.deg)
el = Angle(start_el, unit=u.deg)
print(f'orig az {az} and el {el}')
time_data = await attcs.atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
#print(curr_time_atptg)
coord_frame_AltAz = AltAz(location=location, obstime=curr_time_atptg)
coord_frame_radec = ICRS()
coord_azel = AltAz(az=az, alt=el, location=location, obstime=curr_time_atptg)
ra_dec = coord_azel.transform_to(coord_frame_radec)
print('Current Position is: \n {}'.format(coord_azel))
print('Current Position is: \n {}'.format(ra_dec))

# get RA/DEC of target position
az = Angle(start_az+d_az, unit=u.deg)
el = Angle(start_el+d_el, unit=u.deg)
print(f'target az {az} and el {el}')
coord_azel_target = AltAz(az=az, alt=el, location=location, obstime=curr_time_atptg)
ra_dec_target = coord_azel_target.transform_to(coord_frame_radec)
print('Target Position is: \n {}'.format(coord_azel_target))
print('Target Position is: \n {}'.format(ra_dec_target))


orig az 50.0 deg and el 20.0 deg
Current Position is: 
 <AltAz Coordinate (obstime=58924.32528101147, location=(1819093.56876225, -5208411.6827961, -3195180.61110659) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (50., 20.)>
Current Position is: 
 <ICRS Coordinate: (ra, dec) in deg
    (270.54903191, 20.46133049)>
target az 140.0 deg and el 80.0 deg
Target Position is: 
 <AltAz Coordinate (obstime=58924.32528101147, location=(1819093.56876225, -5208411.6827961, -3195180.61110659) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (140., 80.)>
Target Position is: 
 <ICRS Coordinate: (ra, dec) in deg
    (228.34246068, -37.58622495)>
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 23 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 22 messages


In [12]:
# Why does doing no slew at all take ~32.7 sec?

#send to starting position
await attcs.slew_icrs(ra=str(ra_dec.ra), dec=str(ra_dec.dec), rot_pa=0.0, slew_timeout=240., stop_before_slew=True)


print('track for 2s')
await asyncio.sleep(2)
# take a quick image
await latiss.take_engtest(exptime=1)

print('Starting to Slew to target')
start_time = time.time()
await attcs.slew_icrs(ra=str(ra_dec_target.ra), dec=str(ra_dec_target.dec), rot_pa=d_rot, slew_timeout=240., stop_before_slew=False, wait_settle=False)
end_time = time.time()
print('Time to slew is {} [s]'.format(end_time-start_time))
await latiss.take_engtest(exptime=1)

Parallactic angle: -135.22149995393687 | Sky Angle: -115.18608548092668
Sending command
Stop tracking.
Mount tracking state is 9
Tracking state: 10.
Tracking state: 8.
In Position: True.
In Position: False.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
[Telescope] delta Alt = +000.164 | delta Az = -000.131 delta N1 = +000.000 delta N2 = -000.314 [Dome] delta Az = +003.060
[Telescope] delta Alt = -000.001 | delta Az = +000.002 delta N1 = +000.000 delta N2 = +000.001 [Dome] delta Az = +003.060
Axes in position.
None
track for 2s
Generating group_id
ENGTEST 0001 - 0001
Starting to Slew to target
Parallactic angle: -43.06828420825884 | Sky Angle: 127.00338694947561
Sending command
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>

array([2020031500138])

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 16 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 60 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 18 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 14 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 19 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 19 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


In [16]:
await attcs.stop_tracking()

Stop tracking.
Mount tracking state is 9
Tracking state: 10.
Tracking state: 8.
In Position: True.
In Position: False.
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


In [22]:
# Perform tracking test by taking continous 20s exposures
#await attcs.slew_object('HD 167060')
await attcs.slew_object('WD 1327-083')

Resetting dropped connection: simbad.u-strasbg.fr
http://simbad.u-strasbg.fr:80 "POST /simbad/sim-script HTTP/1.1" 200 None
Slewing to WD 1327-083: 13 30 13.6369 -08 34 29.470
RemoteEvent(ATMCS, 0, target) falling behind; read 27 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 27 messages
Parallactic angle: 129.24008215782175 | Sky Angle: 181.3509863228944
Sending command
Stop tracking.
Mount tracking state is 9
Tracking state: 10.
Tracking state: 8.
In Position: True.
In Position: False.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
[Telescope] delta Alt = +002.333 | delta Az = +144.988 delta N1 = +000.000 delta N2 = +008.465 [Dome] delta Az = +149.742
[Telescope] delta Alt = +000.068 | delta Az = +148.549 delta N1 = +000.000 delta N2 = +003.868 [Dome

In [23]:
await latiss.take_object(exptime=15, grating='empty_1', filter='RG610')

Generating group_id
OBJECT 0001 - 0001


array([2020031500209])

In [19]:
await latiss.take_object(exptime=15, grating='ronchi90lpmm', filter='empty_1')

Generating group_id
OBJECT 0001 - 0001


array([2020031500162])

In [24]:
while True:
    await latiss.take_object(exptime=15, grating='empty_1', filter='RG610')
    await latiss.take_object(exptime=15, grating='ronchi90lpmm', filter='empty_1')

Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 

CancelledError: 

In [27]:
# stopped tracking elsewhere, then reslewed
await attcs.slew_object('WD 1327-083')

Slewing to WD 1327-083: 13 30 13.6369 -08 34 29.470
Parallactic angle: 124.77078877819811 | Sky Angle: 170.14067946777385
Sending command
Stop tracking.
Mount tracking state is 8
In Position: False.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
[Telescope] delta Alt = -000.453 | delta Az = -000.433 delta N1 = -000.000 delta N2 = -009.909 [Dome] delta Az = -010.237
[Telescope] delta Alt = +000.002 | delta Az = +000.002 delta N1 = +000.000 delta N2 = -005.425 [Dome] delta Az = -007.747
[Telescope] delta Alt = +000.002 | delta Az = +000.002 delta N1 = -000.000 delta N2 = -000.639 [Dome] delta Az = -003.827
[Telescope] delta Alt = +000.002 | delta Az = +000.002 delta N1 = +000.000 delta N2 = -000.002 [Dome] delta Az = -002.537
Axes in position.
None


In [30]:
# OFFSET  AZ/EL
await attcs.offset_azel(az=-10*60*60, el=0)

Applying Az/El offset: -36000/ 0 
Waiting for telescope to settle.
Done


In [31]:
# again
await attcs.offset_azel(az=-20*60*60, el=0)

Applying Az/El offset: -72000/ 0 
Waiting for telescope to settle.
Done


In [32]:
# OFFSET  AZ/EL
await attcs.offset_azel(az=-10*60*60, el=0)

Applying Az/El offset: -36000/ 0 
Waiting for telescope to settle.
Done


In [33]:
# OFFSET  AZ/EL
await attcs.offset_azel(az=-5*60*60, el=0)
await attcs.offset_azel(az=-2*60*60, el=0)
await attcs.offset_azel(az=0, el=0)

Applying Az/El offset: -18000/ 0 
Waiting for telescope to settle.
Done
Applying Az/El offset: -7200/ 0 
Waiting for telescope to settle.
Done
Applying Az/El offset: 0/ 0 
Waiting for telescope to settle.
Done


In [34]:
while True:
    await latiss.take_object(exptime=15, grating='empty_1', filter='RG610')
    await latiss.take_object(exptime=15, grating='ronchi90lpmm', filter='empty_1')
    

Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 20 messages
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 0001 - 0001
Generating group_id
OBJECT 

CancelledError: 